In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import json
from itertools import product
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data_types = ['stat', 'nonstat', 'stat_stocklike', 'nonstat_stocklike']
causal_strengths = [.2, .5, 1.]
n_vars = [5, 10]
s_gs = [.5, 1.]
s_bs = [.0, .1, .5, 1.]  #  2., 5.

methods = {
    'Granger': ['time', 'freq'], 
    'PCMCI': ['time'],
    'CCM-Filtering': ['time'],
    'RCV-VarLiNGAM': ['time'],
    'Rhino': ['time'],
    'Dynotears': ['time'],
    'PCMCIomega': ['time'],
    # 'Geweke': ['time'],
}

In [ ]:
def m2label(m_tf):
    return {
        'Granger_time': '\\textbf{Granger}',
        'Granger_freq': '\\textbf{PLACy}',
        'PCMCI_time': '\\textbf{PCMCI}',
        'CCM-Filtering_time': '\\textbf{CCM-Filtering}',
        'RCV-VarLiNGAM_time': '\\textbf{RCV-VarLiNGAM}',
        'Rhino_time': '\\textbf{Rhino}',
        'Dynotears_time': '\\textbf{DYNOTEARS}',
        'PCMCIomega_time': '\\textbf{PCMCI}$_{\\boldsymbol{\\Omega}}$',
        'Geweke_time': '\\textbf{Geweke}',
    }[m_tf]

In [ ]:
def find_max_index(means, stds):
    max_mean = max(means)
    max_indices = [i for i, m in enumerate(means) if m == max_mean]
    if len(max_indices) == 0:
        return []
    if len(max_indices) == 1:
        return [max_indices[0]]
    else:
        min_std = min(stds[i] for i in max_indices)
        result = [i for i in max_indices if stds[i] == min_std]
        return result

In [ ]:
def make_table_short(n_var, s_g):
    todo_dyno = list()
    todo_rhino = list()
    todo_other = list()

    zero_division = np.nan
    methods_tf = [f'{method}_{tf}' for method in methods for tf in methods[method]]
    columns = [m2label(m_tf) for m_tf in methods_tf]
    index = pd.MultiIndex.from_tuples(product(data_types, causal_strengths), names=['', '\\textbf{C}'])
    df = pd.DataFrame(index=index, columns=columns)

    for causal_strength in causal_strengths:
        for data_type in data_types:
            for c in methods_tf:
                method, tf = c.split('_')
                f1 = list()
                for s_b in s_bs:
                    for seed in range(10 if method == 'Rhino' else 100):
                        fname = f'results/{method}_{data_type}/n_vars={n_var}/causal_strength={causal_strength}/s_g={s_g}/s_b={s_b}/seed={seed}.json'
                        try:
                            with open(fname, 'r') as f:
                                data = json.load(f)
                            cm_est_tf = np.asarray(data[f'cm_est_{tf}']).ravel()
                            cm_true = np.asarray(data['cm']).ravel()
                            if causal_strength != 0:
                                f1.append(f1_score(cm_true, cm_est_tf, zero_division=zero_division))
                        except FileNotFoundError:
                            stationary = 'nonstat' not in data_type
                            stock_like = 'stocklike' in data_type
                            x = f'{n_var} {causal_strength} {s_g} {s_b} {seed} {stationary} {stock_like} {method}'
                            if method == 'Rhino':
                                todo_rhino.append(x)
                            elif method == 'Dynotears':
                                todo_dyno.append(x)
                            else:
                                if method != 'CCM-Filtering' or n_var != 10:
                                    todo_other.append(x)
                if len(f1) == 0:
                    x = np.nan
                else:
                    mean, std = np.nanmean(f1), np.nanstd(f1)
                    x = f'{mean:.2f}{{\\tiny$\pm${std:.2f}}}'
                    # x = f'{mean:.2f} $\pm$ {std:.2f}'
                df.loc[(data_type, causal_strength), m2label(c)] = x
    
    for index, row in df.iterrows():
        means = list(map(lambda x: float(x.split('{')[0]) if isinstance(x, str) else x, row.values))
        std = list(map(lambda x: float(x.split('m$')[1][:-1]) if isinstance(x, str) else x, row.values))
        max_indices = find_max_index(means, std)
        for max_index in max_indices:
            best = row.values[max_index]
            c = columns[row.values.tolist().index(best)]
            best = '\\textbf{' + best[:4] + '}' + best[4:]
            df.loc[index, c] = best

    s = df.to_latex(na_rep='---', float_format="%.2f", caption=f'F1 Score - $N={n_var}, \ \sigma_g={s_g}$', label=f'tab:N{n_var}sigmag{s_g}')
    s = s[:13] + '\n\\renewcommand{\\arraystretch}{2}\centering\n\scriptsize' + s[13:]
    s = s.replace('00000', '').replace('llllllllll', 'cccccccccc')
    s = s.replace('\multirow[t]{3}{*}{stat}', '\multirow{3}{*}{\\rotatebox{90}{${\\rm{OU}}(\\sigma_g^m = 0)$}}')
    s = s.replace('\multirow[t]{3}{*}{stat_stocklike}', '\multirow{3}{*}{\\rotatebox{90}{${\\rm{OU}}(\\sigma_g^m > 0)$}}')
    s = s.replace('\multirow[t]{3}{*}{nonstat}', '\multirow{3}{*}{\\rotatebox{90}{$\\widehat{{\\rm{OU}}}(\\sigma_g^m = 0)$}}')
    s = s.replace('\multirow[t]{3}{*}{nonstat_stocklike}', '\multirow{3}{*}{\\rotatebox{90}{$\\widehat{{\\rm{OU}}}(\\sigma_g^m > 0)$}}')
    s = s.replace('\multirow[t]', '\\multirow[c]')
    s = s.replace('\\cline{1-10}', '\\cmidrule(lr){1-10}')
    s = s.replace('\\cmidrule(lr){1-10}\n\\bottomrule', '\\bottomrule')
    os.makedirs('tables', exist_ok=True)
    with open(f'tables/results_n_var={n_var}_s_g={s_g}_short.tex', 'w') as f:
        f.write(s)
    return todo_rhino, todo_dyno, todo_other, df

In [ ]:
_ = make_table_short(5, 1.)

In [ ]:
def make_table(n_var, s_g):
    todo_dyno = list()
    todo_rhino = list()
    todo_other = list()

    zero_division = np.nan
    methods_tf = [f'{method}_{tf}' for method in methods for tf in methods[method]]
    columns = [m2label(m_tf) for m_tf in methods_tf]
    index = pd.MultiIndex.from_tuples(product(data_types, causal_strengths, s_bs), names=['Dataset', 'C', '$\sigma_b$'])
    df = pd.DataFrame(index=index, columns=columns)

    for causal_strength in causal_strengths:
        for s_b in s_bs:
            for data_type in data_types:
                for c in methods_tf:
                    method, tf = c.split('_')
                    f1 = list()
                    for seed in range(10 if method == 'Rhino' else 100):
                        fname = f'results/{method}_{data_type}/n_vars={n_var}/causal_strength={causal_strength}/s_g={s_g}/s_b={s_b}/seed={seed}.json'
                        try:
                            with open(fname, 'r') as f:
                                data = json.load(f)
                            cm_est_tf = np.asarray(data[f'cm_est_{tf}']).ravel()
                            cm_true = np.asarray(data['cm']).ravel()
                            if causal_strength != 0:
                                f1.append(f1_score(cm_true, cm_est_tf, zero_division=zero_division))
                        except FileNotFoundError:
                            stationary = 'nonstat' not in data_type
                            stock_like = 'stocklike' in data_type
                            x = f'{n_var} {causal_strength} {s_g} {s_b} {seed} {stationary} {stock_like} {method}'
                            if method == 'Rhino':
                                todo_rhino.append(x)
                            elif method == 'Dynotears':
                                todo_dyno.append(x)
                            else:
                                if method != 'CCM-Filtering' or n_var != 10:
                                    todo_other.append(x)
                    if len(f1) == 0:
                        x = np.nan
                    else:
                        mean, std = np.nanmean(f1), np.nanstd(f1)
                        x = f'{mean:.2f}{{\\tiny$\pm${std:.2f}}}'
                        # x = f'{mean:.2f} $\pm$ {std:.2f}'
                    df.loc[(data_type, causal_strength, s_b), m2label(c)] = x
    
    for index, row in df.iterrows():
        means = list(map(lambda x: float(x.split('{')[0]) if isinstance(x, str) else x, row.values))
        std = list(map(lambda x: float(x.split('m$')[1][:-1]) if isinstance(x, str) else x, row.values))
        max_indices = find_max_index(means, std)
        for max_index in max_indices:
            best = row.values[max_index]
            c = columns[row.values.tolist().index(best)]
            best = '\\textbf{' + best[:4] + '}' + best[4:]
            df.loc[index, c] = best

    s = df.to_latex(na_rep='---', float_format="%.2f", caption=f'F1 Score - $N={n_var}, \ \sigma_g={s_g}$', label=f'tab:N{n_var}sigmag{s_g}')
    s = s[:13] + '\n\centering\n\scriptsize' + s[13:]
    s = s.replace('00000', '').replace('llllllllll', 'cccccccccc')
    s = s.replace('\cline{1-11} \cline{2-11}', '\midrule')
    s = s.replace('\multirow[t]{12}{*}{stat}', '\multirow{12}{*}{\\rotatebox{90}{${\\rm{OU}}(\\sigma_g^m = 0)$}}')
    s = s.replace('\multirow[t]{12}{*}{stat_stocklike}', '\multirow{12}{*}{\\rotatebox{90}{${\\rm{OU}}(\\sigma_g^m > 0)$}}')
    s = s.replace('\multirow[t]{12}{*}{nonstat}', '\multirow{12}{*}{\\rotatebox{90}{$\\widehat{{\\rm{OU}}}(\\sigma_g^m = 0)$}}')
    s = s.replace('\multirow[t]{12}{*}{nonstat_stocklike}', '\multirow{12}{*}{\\rotatebox{90}{$\\widehat{{\\rm{OU}}}(\\sigma_g^m > 0)$}}')
    s = s.replace('\multirow[t]', '\\multirow[c]')
    s = s.replace('\\midrule\n\\bottomrule', '\\bottomrule')
    s = s.replace('\\cline{2-11}', '\\cmidrule(lr){2-11}')
    os.makedirs('tables', exist_ok=True)
    with open(f'tables/results_n_var={n_var}_s_g={s_g}.tex', 'w') as f:
        f.write(s)
    return todo_rhino, todo_dyno, todo_other, df

In [ ]:
todo_rhino1, todo_dyno1, todo_other1, df1 = make_table(5, .5)
todo_rhino2, todo_dyno2, todo_other2, df2 = make_table(5, 1.)
todo_rhino3, todo_dyno3, todo_other3, df3 = make_table(10, 0.5)
todo_rhino4, todo_dyno4, todo_other4, df4 = make_table(10, 1.)

In [ ]:
todo_rhino = todo_rhino1 + todo_rhino2 + todo_rhino3 + todo_rhino4
todo_dyno = todo_dyno1 + todo_dyno2 + todo_dyno3 + todo_dyno4
todo_other = todo_other1 + todo_other2 + todo_other3 + todo_other4
len(todo_rhino), len(todo_dyno), len(todo_other)

In [ ]:
to_run = 'other'
todo = todo_other
len(todo)

In [ ]:
chunk_size = 1000
for i in range(0, len(todo), chunk_size):
    chunk = todo[i:i + chunk_size]
    with open(f'params_{i // chunk_size + 1}.txt', 'w') as f:
        for item in chunk:
            f.write(f"{item}\n")

In [ ]:
job_name = 'dyno' if to_run == 'dyno' else 'CD'
sif_name = 'causalnex_latest.sif' if to_run == 'dyno' else 'python_cd.sif'
for i in range(len(todo) // chunk_size + 1):
    with open(f'job_{i + 1}.sh', 'w') as f:
        f.write(
f'''#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --output=logs/output_%A_%a.out
#SBATCH --error=logs/error_%A_%a.err
#SBATCH --partition=department_only,students
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --gpus=0
#SBATCH --mincpus=1

params=($(sed -n "${{SLURM_ARRAY_TASK_ID}}p" params_{i+1}.txt))

n_vars=${{params[0]}}
causal_strength=${{params[1]}}
s_g=${{params[2]}}
s_b=${{params[3]}}
seed=${{params[4]}}
stationary=${{params[5]}}
stock_like=${{params[6]}}
method=${{params[7]}}

singularity exec {sif_name} python src/run.py --n_vars $n_vars --causal_strength $causal_strength --s_g $s_g --s_b $s_b --seed $seed --stationary $stationary --stock_like $stock_like --method $method
'''
)

In [ ]:
# Rhino
l = list()
for x in todo_rhino2:
    n_var, causal_strength, s_g, s_b, seed, stationary, stock_like, method = x.split()
    stationary = stationary == 'True'
    stock_like = stock_like == 'True'
    data_type = 'stat' if stationary else 'nonstat'
    if stock_like:
        data_type += '_stocklike'
    l.append(f'n_vars={n_var}_causal_strength={causal_strength}_s_g={s_g}_s_b={s_b}_{data_type}_seed={seed}')
with open(f'data_rhino.txt', 'w') as f:
    for item in l:
        f.write(f"{item}\n")

In [ ]:
from sklearn.metrics import confusion_matrix
def tnr_score(y_true, y_pred):
    tn, fp, _, _ = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    return tn / (tn + fp)

f1 = list()
tnr = list()
for seed in range(17):
    with open(f'results/PCMCIomega_AirQuality/seed={seed}.json', 'r') as f:
        data = json.load(f)
    cm = np.asarray(data['cm']).ravel()
    cm_est_time = np.asarray(data['cm_est_time']).ravel()
    f1.append(f1_score(cm, cm_est_time, zero_division=np.nan))
    tnr.append(tnr_score(cm, cm_est_time))
print('PCMCIomega on AirQuality')
print('F1:\t', np.mean(f1).round(2), ' pm ', np.std(f1).round(2))
print('TNR:\t', np.mean(tnr).round(2), ' pm ', np.std(tnr).round(2))

In [ ]:
from sklearn.metrics import confusion_matrix
def tnr_score(y_true, y_pred):
    tn, fp, _, _ = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    return tn / (tn + fp)

n_var = 5
s_g = 1.
todo = list()
for causal_strength in causal_strengths:
    print(f'C: {causal_strength}')
    for data_type in data_types:
        method, tf = 'Geweke', 'time'
        f1 = list()
        tnr = list()
        for s_b in s_bs:
            for seed in range(100):
                fname = f'results/{method}_{data_type}/n_vars={n_var}/causal_strength={causal_strength}/s_g={s_g}/s_b={s_b}/seed={seed}.json'
                try:
                    with open(fname, 'r') as f:
                        data = json.load(f)
                    cm_est_tf = np.asarray(data[f'cm_est_{tf}']).ravel()
                    cm_true = np.asarray(data['cm']).ravel()
                    if causal_strength != 0:
                        f1.append(f1_score(cm_true, cm_est_tf, zero_division=np.nan))
                        tnr.append(tnr_score(cm_true, cm_est_tf))
                except FileNotFoundError:
                    stationary = 'nonstat' not in data_type
                    stock_like = 'stocklike' in data_type
                    x = f'{n_var} {causal_strength} {s_g} {s_b} {seed} {stationary} {stock_like} {method}'
                    todo.append(x)
        if len(f1) == 0:
            x = np.nan
            y = np.nan
        else:
            f1_mean, f1_std = np.nanmean(f1), np.nanstd(f1)
            tnr_mean, tnr_std = np.nanmean(tnr), np.nanstd(tnr)
            x = f'{f1_mean:.2f} $\pm$ {f1_std:.2f}'
            y = f'{tnr_mean:.2f} $\pm$ {tnr_std:.2f}'
        print(f'data_type: {data_type} F1:{x}  TNR:{y}')
    print()